In [ ]:
### colab 사용 시 실행
from google.colab import drive
drive.mount('/content/gdrive/')
import os
os.chdir('gdrive/My Drive/Google Colaboratory/kakao_arena/kakao_arena_melon/src/ipynb')

In [ ]:
import os
os.chdir('./../../')
from src.basic_utils import *
from src.modeling_utils import *
from src.pivot_utils import *

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data_path = 'data/'

train = pd.read_parquet(data_path+'train(song_trim).parquet')
val = pd.read_json(data_path+'val(lower).json')
# test = pd.read_json(data_path+'test(lower).json')
# song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
# tag_song_dict = load_json(data_path+'tag_song_dict.json')
song_tag_dict = load_json(data_path+'song_tag_dict.json')

In [ ]:
no_tag, no_song, yes_index, no_both = check_target_type(val)

In [ ]:
v1v3_index = no_tag + yes_index

train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))
train['date'] = train['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))
val['date'] = val['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )

year_list = list(train['year'].unique())
year_list= sorted(year_list)

v1v3 = val[val.index.isin(v1v3_index)]

small_years =  [14]
val_tmp = v1v3[v1v3['year'].isin(small_years)]
print("val year shape", val_tmp.shape)

In [ ]:
date_list = sorted(list(train['date'].unique()))
date_dict = {}
for i,k in enumerate(date_list):
    date_dict[k] = i

popular_date_tag = make_popular_date_dict('tags', train, date_list, 13)
popular_date_song = make_popular_date_dict('songs', train, date_list, 9)      

In [ ]:
tmp = train[(train['year'].isin(small_years))]
like_num = int(np.percentile(tmp['like_cnt'], 70))
tmp = tmp[(tmp['like_cnt'] > like_num)]
print("train best playlist", tmp.shape)
print('train+valid = ', tmp.shape[0]+val_tmp.shape[0])

In [ ]:
before_tmp = before_pivot(tmp)

## train만 1보다 작은 노래는 없애준다.
tmp_dict = before_tmp['song'].value_counts().to_dict()
before_tmp['count']= before_tmp['song'].apply(lambda x : tmp_dict[x])
before_tmp = before_tmp[before_tmp['count'] > 1]
print(before_tmp.shape)

before_val_tmp = before_pivot(val_tmp)

In [ ]:
final_result = pd.concat([before_tmp, before_val_tmp])
del before_tmp, before_val_tmp
gc.collect()

In [ ]:
cf_dic = make_cosine_predict(final_result)

In [ ]:
v1v3_predict = pred_v1v3_cosine(val_tmp, song_tag_dict, popular_date_tag)
write_json(v1v3_predict, data_path+'v1v3_'+str(small_years)+'.json')